In [1]:
%%bash
chia version
cdv --version
python --version

1.6.1rc3.dev49
cdv, version 1.1.2
Python 3.10.8


In [2]:
# chia libraries
from blspy import (PrivateKey, AugSchemeMPL, G1Element, G2Element)

from chia.consensus.default_constants import DEFAULT_CONSTANTS
from chia.types.blockchain_format.coin import Coin
from chia.types.blockchain_format.program import Program
from chia.types.coin_spend import CoinSpend
from chia.types.condition_opcodes import ConditionOpcode
from chia.types.spend_bundle import SpendBundle
from chia.util.hash import std_hash
from chia.wallet.puzzles import p2_delegated_puzzle_or_hidden_puzzle

from clvm.casts import int_to_bytes
from clvm_tools.clvmc import compile_clvm_text
from clvm_tools.binutils import disassemble

# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../../../shared")
from utils import (load_program, print_program, print_puzzle, print_json, print_push_tx_result)


## Puzzle Announcement
#### `CREATE_PUZZLE_ANNOUNCEMENT`
`(62 message)`
If this spend is valid, this creates an ephemeral announcement with an ID dependent on the puzzle that creates it. Other coins can then assert an announcement exists for inter-coin communication inside a block.

#### `ASSERT_PUZZLE_ANNOUNCEMENT`
`(63 announcementID)`
This spend is only valid if there was an announcement in this block matching the announcementID. The announcementID is the message that was announced concatenated with the puzzle hash of the coin that announced it `announcementID == sha256(puzzle_hash + message)`.

In [11]:
announcer_puzzle = Program(
    compile_clvm_text(
'''
(mod (N announcement)
    (list
        (list 62 announcement)
    )
)
''', search_paths=[]
    )
)

asserter_puzzle = Program(
    compile_clvm_text(
'''
(mod (announcer_ph announcement)
    (list
        (list 63 (sha256 announcer_ph announcement))
    )
)
''', search_paths=[]
    )
)

print_program(announcer_puzzle)
print_program(asserter_puzzle)

(c (c (q . 62) (c 5 ())) ())
(c (c (q . 63) (c (sha256 2 5) ())) ())


In [12]:
parent_coin_info = bytes.fromhex("f85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1")

announcer_puzzle_1 = announcer_puzzle.curry(1)

announcer_coin = Coin(
    parent_coin_info,
    announcer_puzzle_1.get_tree_hash(),
    1
)

message = bytes("hello", "utf-8")
announcement = std_hash(message)
solution = Program.to([announcement])

result = announcer_puzzle_1.run(solution)
print_program(result)

announcer_coin_spend = CoinSpend(
    announcer_coin,
    announcer_puzzle_1,
    solution
)

print_json(announcer_coin_spend.to_json_dict())

((62 0x2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824))
{
    "coin": {
        "amount": 1,
        "parent_coin_info": "0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1",
        "puzzle_hash": "0xf8814f3c924d9befa37b02c6faf8ca28fe25e7b982dde639194bb5fcaf869b28"
    },
    "puzzle_reveal": "0xff02ffff01ff04ffff04ffff013effff04ff05ff808080ff8080ffff04ffff0101ff018080",
    "solution": "0xffa02cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b982480"
}


In [13]:
asserter_coin = Coin(
    parent_coin_info,
    asserter_puzzle.get_tree_hash(),
    1
)

message = bytes("hello", "utf-8")
announcement = std_hash(message)
solution = Program.to([announcer_puzzle_1.get_tree_hash(), announcement])

result = asserter_puzzle.run(solution)
print_program(result)

asserter_coin_spend = CoinSpend(
    asserter_coin,
    asserter_puzzle,
    solution
)

print_json(asserter_coin_spend.to_json_dict())

((63 0xd0efbccf8c133e5230863790adeb4880af5e842fd4b2b47f4e4889d38a09578a))
{
    "coin": {
        "amount": 1,
        "parent_coin_info": "0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1",
        "puzzle_hash": "0x4e0e250d90fe525477bf182325ffcc5f1d0b692461ebf33f6ff54e3d4d732903"
    },
    "puzzle_reveal": "0xff04ffff04ffff013fffff04ffff0bff02ff0580ff808080ff8080",
    "solution": "0xffa0f8814f3c924d9befa37b02c6faf8ca28fe25e7b982dde639194bb5fcaf869b28ffa02cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b982480"
}


In [14]:
spend_bundle = SpendBundle(
    [
        announcer_coin_spend,
        asserter_coin_spend
    ],
    G2Element()
)

print_json(spend_bundle.to_json_dict())

{
    "aggregated_signature": "0xc00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
    "coin_solutions": [
        {
            "coin": {
                "amount": 1,
                "parent_coin_info": "0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1",
                "puzzle_hash": "0xf8814f3c924d9befa37b02c6faf8ca28fe25e7b982dde639194bb5fcaf869b28"
            },
            "puzzle_reveal": "0xff02ffff01ff04ffff04ffff013effff04ff05ff808080ff8080ffff04ffff0101ff018080",
            "solution": "0xffa02cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b982480"
        },
        {
            "coin": {
                "amount": 1,
                "parent_coin_info": "0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1",
                "puzzle_hash": "0x4e0e250d90fe525477bf182325ffcc5f1d0b692461ebf3

### Debug Spend Bundle
❯ cdv inspect spendbundles $spendbundle -db
...
Debugging Information
---------------------
================================================================================
consuming coin (0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1 0xf8814f3c924d9befa37b02c6faf8ca28fe25e7b982dde639194bb5fcaf869b28 1)
  with id fa2c07b885b795659c271466d378c67e388753c58a34186b7a70fa4eda6c5f9b


brun -y main.sym '(a (q 4 (c (q . 62) (c 5 ())) ()) (c (q . 1) 1))' '(0x2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824)'

((CREATE_PUZZLE_ANNOUNCEMENT 0x2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824))

grouped conditions:

  (CREATE_PUZZLE_ANNOUNCEMENT 0x2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824)


-------
consuming coin (0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1 0x4e0e250d90fe525477bf182325ffcc5f1d0b692461ebf33f6ff54e3d4d732903 1)
  with id be0a385c46c20f772ca55d82646f7c49481eb83390edb09b68e2366fd29ec30f


brun -y main.sym '(c (c (q . 63) (c (sha256 2 5) ())) ())' '(0xf8814f3c924d9befa37b02c6faf8ca28fe25e7b982dde639194bb5fcaf869b28 0x2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824)'

((ASSERT_PUZZLE_ANNOUNCEMENT 0xd0efbccf8c133e5230863790adeb4880af5e842fd4b2b47f4e4889d38a09578a))

grouped conditions:

  (ASSERT_PUZZLE_ANNOUNCEMENT 0xd0efbccf8c133e5230863790adeb4880af5e842fd4b2b47f4e4889d38a09578a)


-------

spent coins
  (0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1 0x4e0e250d90fe525477bf182325ffcc5f1d0b692461ebf33f6ff54e3d4d732903 1)
      => spent coin id be0a385c46c20f772ca55d82646f7c49481eb83390edb09b68e2366fd29ec30f
  (0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1 0xf8814f3c924d9befa37b02c6faf8ca28fe25e7b982dde639194bb5fcaf869b28 1)
      => spent coin id fa2c07b885b795659c271466d378c67e388753c58a34186b7a70fa4eda6c5f9b

created coins
created puzzle announcements
  ['0xf8814f3c924d9befa37b02c6faf8ca28fe25e7b982dde639194bb5fcaf869b28', '0x2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824'] =>
      d0efbccf8c133e5230863790adeb4880af5e842fd4b2b47f4e4889d38a09578a


zero_coin_set = []

created  puzzle announcements = ['d0efbccf8c133e5230863790adeb4880af5e842fd4b2b47f4e4889d38a09578a']

asserted puzzle announcements = ['d0efbccf8c133e5230863790adeb4880af5e842fd4b2b47f4e4889d38a09578a']

symdiff of puzzle announcements = []


================================================================================

aggregated signature check pass: True
pks: []
msgs: []
  msg_data: []
  coin_ids: []
  add_data: []
signature: c00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
None

## Run on Simulator

In [16]:
from cdv.test import Network, Wallet

async def get_network_n_wallets():
    network: Network = await Network.create()
    await network.farm_block()

    alice: Wallet = network.make_wallet("alice")
    bob: Wallet = network.make_wallet("bob")
    await network.farm_block(farmer=alice)
    await network.farm_block(farmer=bob)
    return network, alice, bob

In [24]:
network, alice, bob = await get_network_n_wallets()

# alice's coin is an announcer
alice_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
alice_conditions = [
    [
        ConditionOpcode.CREATE_PUZZLE_ANNOUNCEMENT,
        announcement
    ],

    # return
    [
        ConditionOpcode.CREATE_COIN, 
        alice.puzzle_hash, 
        alice_coin_wrapper.coin.amount
    ]
]

alice_coin_spend, alice_sig = alice_coin_wrapper.create_standard_spend(alice.sk_, alice_conditions)

# bob's coin is an asserter
bob_coin_wrapper = await bob.choose_coin(1_750_000_000_000)
bob_conditions = [
    [
        ConditionOpcode.ASSERT_PUZZLE_ANNOUNCEMENT,
        std_hash(alice.puzzle_hash + announcement)
    ],
    
    # return
    [
        ConditionOpcode.CREATE_COIN, 
        bob.puzzle_hash, 
        bob_coin_wrapper.coin.amount
    ]
]

bob_coin_spend, bob_sig = bob_coin_wrapper.create_standard_spend(bob.sk_, bob_conditions)
spend_bundle = SpendBundle(
    [ alice_coin_spend, bob_coin_spend ],
    AugSchemeMPL.aggregate([alice_sig, bob_sig])
)
result = await network.push_tx(spend_bundle)
#print(result)
print_push_tx_result(result)

additions:
Coin { parent_coin_info: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba, puzzle_hash: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3, amount: 1750000000000 }
Coin { parent_coin_info: 17ca02c0a209d7e1a3869442ba13ef9468181c4b095b8823aeaf3c27f8e58c34, puzzle_hash: 87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5, amount: 1750000000000 }
removals:
Coin { parent_coin_info: e3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001, puzzle_hash: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3, amount: 1750000000000 }
Coin { parent_coin_info: e3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000002, puzzle_hash: 87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5, amount: 1750000000000 }
